In [ ]:
def normalize(x, maks):
    return x / maks
  
def denormalize(x, maks):
    return x * maks

def r1c1_model_flex(R_amb, R_q, R_sun, T_amb_diff, Q, Sun, previous_dTdt_values, resistances, rcOrder):
    dTdt = (T_amb_diff / R_amb) + (Q / R_q) + (Sun / R_sun)  # Initial terms
    
    for i in range(rcOrder):
        dTdt += previous_dTdt_values[i] / resistances[i]  # Adding terms based on rcOrder
    return dTdt

def maximum(a, b):
    if a >= b:
        return a
    else:
        return b 

def nth_root(number, n):
    return math.pow(number, 1/n)
    
def extract_every_nth_row(df, N):
    extracted_df = df.iloc[::N]
    return extracted_df

def create_batches(data, batch_size):
    if len(data) % batch_size != 0:
        raise ValueError("Batch size must be divisible by the length of the data.")
    num_batches = len(data) // batch_size
    batches = []
    for i in range(num_batches):
        batch = data[i * batch_size: (i + 1) * batch_size]
        batches.append(batch)
    return batches
        
def ignore_minimize_step_warning(message, category, filename, lineno, file=None, line=None):
    if "Values in x were outside bounds during a minimize step, clipping to bounds" in str(message):
        return True
    return False

def r1c1_model_flex(R_amb, R_q, R_sun, T_amb_diff, Q, Sun, previous_dTdt_values, resistances, rcOrder):
    dTdt = (T_amb_diff / R_amb) + (Q / R_q) + (Sun / R_sun)  # Initial terms
    
    for i in range(rcOrder):
        dTdt += previous_dTdt_values[i] / resistances[i]  # Adding terms based on rcOrder
    return dTdt

def R1C1_Model_Train_flex(Model_data, dt, trainPredHours, chunkTrainingPercentage, Plot_Modelperformance, prevOutputRArray, rcOrder):
    # Prepare Data
    Q = Model_data['Energy']
    T_amb_diff = Model_data['Ambient_temp_diff']
    Sun = Model_data['Solar Rad']
    T_observed = Model_data['Target_Temp']
    T_amb = Model_data['Ambient_Temp']
    previous_dTdt_values = Model_data[['Change_Target_Temp_shifted_1','Change_Target_Temp_shifted_2','Change_Target_Temp_shifted_3','Change_Target_Temp_shifted_4']]
    trainPredSteps = int(round((trainPredHours / 24) / dt))
    
    # Define the modified cost function to be minimized
    def cost_function(params, dt, T_amb, T_amb_diff, T_observed, Q, Sun, resistances, rcOrder):
        cost = 0
        R_amb, R_q, R_sun, *resistances = params
        
        if R_amb < 0:
            return 1e12
        if R_q < 0:
            return 1e12
        if R_sun < 0:
            return 1e12
        if any(r < 0 for r in resistances):
            return 1e12  
        if rcOrder > 1:
            if np.min(np.diff(resistances[:rcOrder]))<0:
                return 1e12
        
        # Define the number of samples N
        Y = len(T_observed) - (trainPredSteps +1)  # Upper bound
        nPointsDrawn = round(chunkTrainingPercentage * len(T_observed))  # Number of samples to draw

        # Draw N numbers from a uniform distribution within the range [1, Y]
        startingPoints = np.round(np.linspace(1, Y, nPointsDrawn)).astype(int)

        for i in range(nPointsDrawn):
            T_predicted = np.zeros(trainPredSteps)
            T_int = T_observed[startingPoints[i] - 1]
            T_amb_diff_temp = T_amb_diff[startingPoints[i] - 1] 
            previous_dTdt_values_temp = previous_dTdt_values.iloc[startingPoints[i] - 1].values
            for j in range(trainPredSteps):
                dTdt = r1c1_model_flex(R_amb, R_q, R_sun, T_amb_diff_temp, Q[startingPoints[i] + j -1], Sun[startingPoints[i] + j -1], previous_dTdt_values_temp, resistances, rcOrder)
                T_predicted[j] = normalize(denormalize(T_int, max_x) + denormalize(dTdt, max_y), max_x)
                T_int = T_predicted[j]
                T_amb_diff_temp = normalize(denormalize(T_amb[startingPoints[i] + j], max_d[0]) - denormalize(T_int, max_x), max_d[2])
                previous_dTdt_values_temp = np.insert(previous_dTdt_values_temp[:-1], 0, dTdt)

            cost += np.sum((T_observed[startingPoints[i]:(startingPoints[i] + trainPredSteps)] - T_predicted)**2)

        print("\r", cost, end='')
        return cost

    # Initial guesses for C and R
    initial_guess = [2.7334965468888583, 0.5968899368320202, 7.940914892704567] + [1.0, 10.0, 100.0, 1000.0] #typical 0-th order values. Just chosen to reduce training time ;)

    # Perform parameter estimation
    param_Result = minimize(cost_function, initial_guess, args=(dt, T_amb, T_amb_diff, T_observed, Q, Sun, prevOutputRArray, rcOrder), method='Nelder-Mead')

    # Extract the estimated parameters
    R_amb_estimated, R_q_estimated, R_sun_estimated, *resistances = param_Result.x

    print("\r",f"Estimated R_amb: {R_amb_estimated}")
    print(f"Estimated R_q: {R_q_estimated}")
    print(f"Estimated R_sun: {R_sun_estimated}")
    
    for i, resistance_value in enumerate(resistances, start=1):
        print(f"Estimated R_{i}: {resistance_value}")
    print(f"Lowest function value: {param_Result.fun}")
    
    
    if Plot_Modelperformance == True:
        deviationArr = np.zeros(len(predTestArray))
        for k in range(len(predTestArray)):
            trainPredSteps = int(round((predTestArray[k] / 24) / dt))
            
            R1C1_pred = np.zeros(len(Model_data))
            for i in range(len(Model_data) - trainPredSteps -1):
                temp_pred = T_observed[i]
                T_amb_diff_temp = T_amb_diff[i]
                previous_dTdt_values_temp = previous_dTdt_values.iloc[i].values
                for j in range(trainPredSteps):
                    dTdt = r1c1_model_flex(R_amb_estimated, R_q_estimated, R_sun_estimated, T_amb_diff_temp, Q[i + j], Sun[i + j], previous_dTdt_values_temp, resistances, rcOrder)
                    temp_pred = normalize(denormalize(temp_pred, max_x) + denormalize(dTdt, max_y), max_x)
                    T_amb_diff_temp = normalize(denormalize(T_amb[i + j + 1], max_d[0]) - denormalize(temp_pred, max_x), max_d[2])
                    previous_dTdt_values_temp = np.insert(previous_dTdt_values_temp[:-1], 0, dTdt)
                    
                R1C1_pred[i + trainPredSteps + 1] = temp_pred
                
            predict = np.array(R1C1_pred * data_max['Target_Temp']).reshape(1, -1)
            data_output = np.array(T_observed * data_max['Target_Temp']).reshape(1, -1)
            deviationArr[k] = np.mean(np.abs(predict[0, (trainPredSteps+1):] - data_output[0, (trainPredSteps+1):]))
        
        px.line(deviationArr).show()
        
    return param_Result.fun, R_amb_estimated, R_q_estimated, R_sun_estimated, np.array([resistances[0], resistances[1], resistances[2], resistances[3]])


def nRnC_MPC_Cost(u, Next_T, Change_Target_Temp_shifted_vector, Ambient_Temp_Forecast, Solar_Rad_Forecast, T_ref_Vector, T_ref_Lower, R, C, C_delta, H_p, i_off, temp_input, rcOrder):
    cost = 0 
    lower_cost = 0
    delta_u = 0 
    u_normalized = normalize(u, max_u)
    
    for t in range(H_p):
        index = t * downsampleNumber + i_off
        
        # Updating Ambient temp diff
        Ambient_temp_diff = normalize((Ambient_Temp_Forecast[index] - Next_T), max_d[2])
        
        # Predicting next change in temp
        Change_Target_Temp = r1c1_model_flex(R_Ambient_Temp, R_Heat_Pump, R_Solar_Rad, Ambient_temp_diff, u_normalized[t], Solar_Rad_Forecast[index], Change_Target_Temp_shifted_vector, prevOutputRArray, rcOrder)
        
        # Updating Temperatures
        Next_T = denormalize(Change_Target_Temp, max_y) + Next_T
        
        # update previous temps
        Change_Target_Temp_shifted_vector = np.insert(Change_Target_Temp_shifted_vector[:-1], 0, Change_Target_Temp)

        #Lower boundary penalty
        lower_diff = T_ref_Lower[(index+downsampleNumber) % len(T_ref_Lower)] - Next_T
        lower_diff = np.where(lower_diff <= 0, 0, lower_diff)
        lower_diff = (lower_diff) * R * 10
        
        if t == 0:
            if temp_input*5000 < u[t]:
                delta_u = abs(normalize((temp_input*5000 - u[t])**2, max_u_delta))
                
        if t > 0:
            if u[t-1] < u[t]:
                delta_u = abs(normalize((u[t-1] - u[t])**2, max_u_delta))
            
        #Update cost
        cost += (abs(Next_T-T_ref_Vector[(index+downsampleNumber)%len(T_ref_Lower)]))*R + C*u_normalized[t] + lower_diff + C_delta*(delta_u)
        
    return cost


class PINN(nn.Module):
    def __init__(self, inputWidth, layerWidth, layerDepth):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(in_features=inputWidth, out_features=layerWidth))
        for _ in range(layerDepth - 1):
            self.layers.append(nn.Linear(in_features=layerWidth, out_features=layerWidth))
        self.output_layer = nn.Linear(in_features=layerWidth, out_features=1)
        self.activation = nn.Tanh()

    def forward(self, inputs):
        x = inputs
        for layer in self.layers:
            x = self.activation(layer(x))
        x = self.output_layer(x)
        return x
    
    
def PINN_loss_with_BPTT(model, input_data, output_data, condition_data):
    
    # Data loss
    data_loss = torch.tensor(0.0, requires_grad=True)
    for i in range(batch_size_sim-1):
        predicted_output = model(input_data[i,:]) 
        next_temperature = normalize(denormalize(condition_data[i, 0], data_max[0]) + denormalize(predicted_output, data_max[4]), data_max[0])
        amb_diff = normalize(denormalize(condition_data[i,1], max_d[0])-denormalize(next_temperature, max_x), max_d[2])
        loss_term = (predicted_output - output_data[i])**2
        data_loss = data_loss + loss_term
        input_data[i+1, 2].data.copy_(amb_diff.data.item())
            
    return data_loss/batch_size_sim

def PINN_loss_direct(model, input_data, output_data):
    # Physics loss
    physics_loss = torch.tensor(0.0, requires_grad=True)
    for i in range(batch_size_nRnC):
        predicted_output = model(input_data[i,:]) 
        loss_term = (predicted_output - output_data[i])**2
        physics_loss = physics_loss + loss_term

    return physics_loss/batch_size_nRnC


def PINN_MPC_Cost(u, Next_T, Change_Target_Temp_shifted_vector, Ambient_Temp_Forecast, Solar_Rad_Forecast, T_ref_Vector, T_ref_Lower, R, C, C_delta, H_p, i_off, temp_input):
    cost = 0 
    lower_cost = 0
    delta_u = 0 
    u_normalized = normalize(u, max_u)
    for t in range(H_p):
        index = t * downsampleNumber + i_off
        
        # Updating Ambient temp diff
        Ambient_temp_diff = normalize((Ambient_Temp_Forecast[index] - Next_T), max_d[2])
        
        X = np.concatenate((
                       [Solar_Rad_Forecast[index]], 
                       [u_normalized[t]], 
                       [Ambient_temp_diff]))
        
        for b in range(chosen_model):
            X = np.concatenate((X, [Change_Target_Temp_shifted_vector[b]]),axis=0)
            
        X_tensor = torch.tensor(X)
        X_tensor = X_tensor.float()
        
        # Predicting next change in temp
        Change_Target_Temp = model(X_tensor)
        
        # Updating Temperatures
        Next_T = denormalize(Change_Target_Temp.item(), max_y) + Next_T
        #print(u[t])
        #print(denormalize(Change_Target_Temp.item(), max_y))
        # update previous temps
        Change_Target_Temp_shifted_vector = np.insert(Change_Target_Temp_shifted_vector[:-1], 0, Change_Target_Temp.item())

        #Lower boundary penalty
        lower_diff = T_ref_Lower[(index+downsampleNumber) % len(T_ref_Lower)] - Next_T
        lower_diff = np.where(lower_diff <= 0, 0, lower_diff)
        lower_diff = (lower_diff) * R * 10
        
        if t == 0:
            delta_u = normalize((max((u[t]-temp_input*5000),0))**2, max_u_delta)
        if t > 0:
            delta_u = normalize((max((u[t] - u[t-1]),0))**2, max_u_delta)
            
        #Update cost
        cost += (abs(Next_T-T_ref_Vector[(index+downsampleNumber)%len(T_ref_Lower)]))*R + C*u_normalized[t] + C_delta*(delta_u) + lower_diff
             
    return cost

def KPIculculater(out_df_MPC, T_ref_Lower, T_ref_Upper):
    
    time_in_range = 0
    total_time = len(out_df_MPC)
    
    TIR_Lower = np.tile(T_ref_Lower, paramTuningInterval)
    TIR_Upper = np.tile(T_ref_Upper, paramTuningInterval)
    out_df_MPC_RoomTemp = np.array([out_df_MPC['temRoo.T']]).reshape(-1,1)
    out_df_MPC_Ambient = np.array([out_df_MPC['TOut.T']]).reshape(-1,1)
    out_df_MPC_Energy = np.array([out_df_MPC['heaPum.P']]).reshape(-1,1)
    out_df_MPC_solar = normalize(np.array([out_df_MPC['sunRad.y']]).reshape(-1,1), max_d[1])
    TIR_Lower = (TIR_Lower - CelsiusToKelvin).reshape(-1,1)
    TIR_Upper = (TIR_Upper - CelsiusToKelvin).reshape(-1,1)
    out_df_MPC_Ambient = out_df_MPC_Ambient - CelsiusToKelvin
    out_df_MPC_RoomTemp = out_df_MPC_RoomTemp - CelsiusToKelvin

    
    values_below_range = 0  #Integer value for how many samples were below the range.
    for i in range(len(TIR_Lower)):
        if (i%288 >= Timestamp1) and (i%288 <= (Timestamp1 + Ref_change_penalty_length)):
            if out_df_MPC_RoomTemp[i] < TIR_Lower[i]:
                values_below_range += 4
        
                
    time_in_range = np.sum(out_df_MPC_RoomTemp >= TIR_Lower)
    percentage_in_range = (time_in_range / total_time) * 100
    
    values_above_range = 0
    for i in range(len(TIR_Lower)):
        if (i%288 >= Timestamp1) and (i%288 <= (Timestamp1 + Ref_change_penalty_length)):
            if out_df_MPC_RoomTemp[i] > TIR_Upper[i]:
                values_above_range += 2

    percentage_in_range_scaled = 1-((percentage_in_range - 80)/(100 - 80))
    EnergyConsumption_scaled = ((out_df_MPC_Energy.reshape(1,-1))/max_u).mean()
    EnergyDelta_scaled = abs(np.diff(out_df_MPC_Energy.reshape(1,-1))/max_u_delta_non_square).mean()

    return percentage_in_range_scaled, EnergyConsumption_scaled, EnergyDelta_scaled, values_below_range/((Ref_change_penalty_length)*paramTuningInterval), values_above_range/((Ref_change_penalty_length)*paramTuningInterval)

def MPC(trial):
    global progress_bar, steps_per_Day, downsampleNumber, paramTuningInterval
    global ChosenPredModel, PINN_MPC_Cost, nRnC_MPC_Cost, u0, Target_Temp
    global Change_Target_Temp_shifted_vector, Ambient_Temp_Forecast, Solar_Rad_Forecast
    global T_ref_Vector, T_ref_Lower, R, H_p, temp_input, u_max, chosen_model
    global control, controls, outputs, env, out_list, data
    global observations
    
    if hyperParamTuner==0:
        C, C_delta = trial#[400,400]#trial
    else:
        C = trial.suggest_float('C', 0.0, 2*R/(2* initial_u))  # Example suggestion for C
        C_delta = trial.suggest_float('C_delta', 0.0, 2*R/(2* initial_u))  # Example suggestion for C_delta

    for j in range(steps_per_Day*downsampleNumber*paramTuningInterval):
        progress_bar.update(1)

        if (j%downsampleNumber == 0):


            if ChosenPredModel == 0:
                res = minimize(PINN_MPC_Cost, u0, args=(
                                Target_Temp, Change_Target_Temp_shifted_vector, Ambient_Temp_Forecast, 
                                Solar_Rad_Forecast, T_ref_Vector, T_ref_Lower, 
                                R, C, C_delta, H_p, j, temp_input
                               ), bounds = u_max, method="Nelder-Mead")
                temp_input = res.x[0]/5000

            else:
                res = minimize(nRnC_MPC_Cost, u0, args=(
                                Target_Temp, Change_Target_Temp_shifted_vector, Ambient_Temp_Forecast, 
                                Solar_Rad_Forecast, T_ref_Vector, T_ref_Lower, 
                                R, C, C_delta, H_p, j, temp_input, chosen_model
                               ), bounds = u_max, method="SLSQP")
                temp_input = res.x[0]/5000

        #take step in simulation with the computed control input       

        control['u'] = [temp_input]
        controls +=[ {p:control[p][0] for p in control} ]
        outputs = env.step(control)
        _,hour,_,_ = env.get_date()
        out_list.append(outputs)
        Target_Temp = out_list[-1]['temRoo.T'] 

        if j % downsampleNumber == 0:
            Change_Target_Temp_shifted_vector = np.insert(Change_Target_Temp_shifted_vector[:-1], 0, normalize(Target_Temp - out_list[-4]['temRoo.T'], max_y))
    
#         if j % downsampleNumber == 0 and j >= downsampleNumber:
#             Change_Target_Temp_shifted_vector = np.insert(Change_Target_Temp_shifted_vector[:-1], 0, normalize(Target_Temp - out_list[-4]['temRoo.T'], max_y))
    
    
    data = pd.DataFrame(out_list[-(steps_per_Day*downsampleNumber*paramTuningInterval):])
    lastDaysData = data.iloc[-(steps_per_Day*downsampleNumber*paramTuningInterval):]
    PIR, Energy, Energy_delta, Ref_rise_penalty, Above_range = KPIculculater(lastDaysData, T_ref_Lower, T_ref_Upper)
    #print(PIR, Energy, Energy_delta, Ref_rise_penalty, Above_range)
    print(C, C_delta, PIR, Ref_rise_penalty, Above_range, Energy, Energy_delta)
    
    if hyperParamTuner==0:
        observations.append(((C, C_delta), (Ref_rise_penalty + Above_range + Energy_delta + PIR)))
    else:
        #observations.append(((C, C_delta), (PIR, Energy, Energy_delta, Ref_rise_penalty, Above_range)))
        observations.append(((C, C_delta), (Energy_delta, Ref_rise_penalty, Above_range)))
    #Return KPIs
    if hyperParamTuner==0:
        return Ref_rise_penalty + Above_range + Energy_delta + PIR #Energy #+ PIR + Above_range
    else:
        return [Energy_delta, Ref_rise_penalty, Above_range]

def Plotting(out_df_MPC, T_ref_Lower_prescaled, T_ref_Vector, Sim_days, title = "Simulated Data"):
    T_ref_Vector = T_ref_Vector - CelsiusToKelvin
    T_ref_Lower_prescaled = T_ref_Lower_prescaled - CelsiusToKelvin
    
    T_ref_Vector_finalsimdays = T_ref_Vector
    
    T_ref_Lower_prescaled_finalsimdays = T_ref_Lower_prescaled
    
    for h in range(Sim_days-1):
        T_ref_Vector_finalsimdays = np.concatenate((T_ref_Vector_finalsimdays, T_ref_Vector))
        T_ref_Lower_prescaled_finalsimdays = np.concatenate((T_ref_Lower_prescaled_finalsimdays, T_ref_Lower_prescaled))
    
#     TotalEnergyConsumption = np.sum(np.array(out_df['heaPum.P']), axis=0)
#     TemperatureDeviation = out_df
#     TemperatureDeviation = np.mean((T_ref_Vector_finalsimdays-TemperatureDeviation)**2)
#     print(f"Temperature deviation: {TemperatureDeviation}, Total Energy Consumption: {TotalEnergyConsumption}")
    
    Y_Axis = (T_ref_Vector_finalsimdays).reshape(1,-1)
    T_ref_Lower_prescaled = (T_ref_Lower_prescaled_finalsimdays).reshape(1,-1)
    columns_temp_MPC = [col for col in out_df_MPC if 'temRoo' in col]
    RoomTemp = out_df_MPC[columns_temp_MPC].copy()
    RoomTemp = RoomTemp.to_numpy()
    RoomTemp = (RoomTemp - CelsiusToKelvin).reshape(1,-1)
    
    columns_temp_MPC = [col for col in out_df_MPC if 'heaPum.P' in col]
    EnergyConsumption = out_df_MPC[columns_temp_MPC].copy()
    EnergyConsumption = EnergyConsumption.to_numpy()
    EnergyConsumption = EnergyConsumption.reshape(1,-1)
    
    out_df_MPC = out_df_MPC.to_numpy()
    Amb = out_df_MPC[:,0]-CelsiusToKelvin
    
    # Plots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(y=RoomTemp[0,:], name="Room Temperature"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(y=Amb, name="Ambient Temperature"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(y=EnergyConsumption[0], name="Energy Consumption"),
        secondary_y=True,
    )
    
    fig.add_trace(
        go.Scatter(y=Y_Axis[0], mode='lines', line=dict(color='black', dash='dot'), name='Temperature Reference'),
        secondary_y=False,
    )

    fig.add_trace(
    go.Scatter(
    y=T_ref_Lower_prescaled[0],
    line=dict(color='rgba(255,255,255,0)'),
    fill='tonexty',
    fillcolor="rgba(0, 255, 0, 0.2)",
    name='Temperature Reference (Lower)',
    showlegend=False,
    ),
    secondary_y=False,
    )
        
    fig.update_layout(
        title_text=title
    )

    fig.update_xaxes(title_text="Steps")
    fig.update_yaxes(title_text="<b>Temperature</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Energy Consumption</b>", secondary_y=True)

    fig.show()
    
def KPIplotting(observations):
    # Get the maximum values of the observations
    max_values = observations.max()
    print(max_values)
    
    # Normalize the observations by dividing by the maximum values
    normalized_df = observations
    
    # Create the figure with two y-axes
    fig = go.Figure()

    # Add traces for the first two parameters on the primary y-axis
    fig.add_trace(go.Scatter(
        x=normalized_df.index,
        y=normalized_df['C'],
        mode='lines',
        name='C',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=normalized_df.index,
        y=normalized_df['C_delta'],
        mode='lines',
        name='C_delta',
        line=dict(color='green')
    ))

    # Add trace for the last parameter on the secondary y-axis
    fig.add_trace(go.Scatter(
        x=normalized_df.index,
        y=normalized_df['KPI'],
        mode='lines',
        name='KPI',
        line=dict(color='red'),
        yaxis='y2'
    ))

    # Update layout for the secondary y-axis
    fig.update_layout(
        yaxis=dict(
            title='Hyperparameter values'
        ),
        yaxis2=dict(
            title='KPI values',
            overlaying='y',
            side='right'
        ),
        xaxis=dict(
            title='Iterations'
        ),
        title='Plot of BO performance'
    )

    # Show the plot
    fig.show()